In [32]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

class Event:
    def __init__(self, start, end, param_multipliers):
        self.start = start
        self.end = end
        self.param_multipliers = param_multipliers  # Multipliers for the parameters

    def is_active(self, t):
        return self.start <= t < self.end

class SEIRSModel:
    def __init__(self, params, events, hospital_capacity):
        self.params = params
        self.events = events
        self.hospital_capacity = hospital_capacity

    def deriv(self, y, t):
        S, E, I, R, D = y
        N = S + E + I + R

        # Start with the base parameters
        current_params = self.params.copy()

        # Adjust parameters based on active events
        for event in self.events:

            if event.is_active(t):
                #print(t)
                # Apply multipliers to the parameters
                for param, multiplier in event.param_multipliers.items():
                    
                    current_params[param] *= multiplier

        # Extract the necessary parameters
        beta, sigma, gamma, delta, mu = current_params['beta'], current_params['sigma'], current_params['gamma'], current_params['delta'], current_params['mu_base']

        # Adjust mortality rate based on hospital capacity
        mu += max(0, (I - self.hospital_capacity) / N * self.params['mu_factor'])

        dSdt = -beta * S * I / N + delta * R
        dEdt = beta * S * I / N - sigma * E
        dIdt = sigma * E - gamma * I - mu * I
        dRdt = gamma * I - delta * R
        dDdt = mu * I
        return dSdt, dEdt, dIdt, dRdt, dDdt

    def simulate(self, y0, t, rtol=1e-3, atol=1e-6):
        results = odeint(self.deriv, y0, t, full_output=0, rtol=rtol, atol=atol)
        return results


# Simulation function
def simulate_seirs_events(model, y0, t):
    return model.simulate(y0, t)
    # results = []

    # for ti in t:

    #     # Solve for the next step
    #     t_end, result = model.simulate(y0, [ti, ti + (t[-1] / 1000)])        
    #     y0 = result[-1]
    #     results.append(y0)

    # results = np.array(results)
    # return t, results

params = {
    'N': 331002647,
    'beta': .2,
    'sigma': 1 / 5,
    'gamma': 1 / 14,
    'delta': 1 / 365,
    'mu_base': 0.001 / 14,  # Base mortality rate per day
    'mu_factor': 0.00  # Additional mortality factor when over capacity
}
hospital_capacity = 2000  # Hospital capacity for serious cases

events = [
    # Lockdowns
    Event(20, 25, {'beta': .5}), 
    Event(25, 30, {'beta': .3}),   
    # Event(50, 70, {'beta': .2}),  
    # Event(70, 100, {'beta': .3}),  
]

for i in range(16):
    events.append(Event(30 + i * 5, 30 + (1+i) * 5, {'beta': .6 + i*.05}), )
print(events)
model = SEIRSModel(params, events, hospital_capacity)


# Initial conditions
I0, R0, E0, D0 = 1, 0, 0, 0
S0 = params['N'] - I0 - E0 - R0 - D0
y0 = S0, E0, I0, R0, D0
t = np.linspace(0, 120, 120)

@widgets.interact
def update(beta=(0.1, 4.0, 0.1), sigma=(0.1, .3, 0.01), gamma=(0.05, 0.2, 0.01), delta=(0.001, 0.01, 0.001), mu_base=(0.0, 0.001, 0.0001)):
    model.params.update({
        'beta': beta,
        'sigma': sigma,
        'gamma': gamma,
        'delta': delta,
        'mu_base': mu_base
    })
    results = model.simulate(y0, t)
    S, E, I, R, D = results.T
    new_I = I
    plt.figure(figsize=(10, 6))
    plt.plot(t, new_I, 'r--', label='New Infections')

    # plt.plot(t, S, label='Susceptible')
    # plt.plot(t, E, label='Exposed')
    # plt.plot(t, I, label='Infectious')
    # plt.plot(t, R, label='Recovered')
    # plt.plot(t, D, label='Deceased')
    plt.title('SEIRS Model Simulation')
    plt.xlabel('Time (days)')
    plt.ylabel('Number of people')
    plt.legend()
    plt.grid(True)
    plt.show()


[<__main__.Event object at 0x00000240BB159AB0>, <__main__.Event object at 0x00000240B985B550>, <__main__.Event object at 0x00000240BB15AF80>, <__main__.Event object at 0x00000240B985A560>, <__main__.Event object at 0x00000240B9859E70>, <__main__.Event object at 0x00000240B9858610>, <__main__.Event object at 0x00000240B9858820>, <__main__.Event object at 0x00000240B9859810>, <__main__.Event object at 0x00000240B985AE60>, <__main__.Event object at 0x00000240B98585B0>, <__main__.Event object at 0x00000240B98581C0>, <__main__.Event object at 0x00000240B9859FF0>, <__main__.Event object at 0x00000240B985B6D0>, <__main__.Event object at 0x00000240B98596F0>, <__main__.Event object at 0x00000240B98599F0>, <__main__.Event object at 0x00000240B9859ED0>, <__main__.Event object at 0x00000240B9858130>, <__main__.Event object at 0x00000240B9858F10>]


interactive(children=(FloatSlider(value=2.0, description='beta', max=4.0, min=0.1), FloatSlider(value=0.2, des…